In [2]:
import psutil
import sys

# Python List Memory Model 

This notebook explores how Python lists manage memory allocation and growth. 

## Key Concepts

**Dynamic Arrays**: Python lists are implemented as dynamic arrays that can grow and shrink during runtime.

**Over-allocation**: Python pre-allocates extra memory slots to minimize the frequency of expensive memory reallocations.

**Amortized O(1) Append**: While individual append operations might trigger expensive reallocations, the average cost remains constant due to the over-allocation strategy.

## Establishing Memory Baseline

Before we start creating large data structures, let's check how much RAM is available on our system. This gives us a baseline to understand the impact of our operations.

In [ ]:
print(f"Available RAM: {psutil.virtual_memory().available / (1024 ** 3):.2f} GB")

Available RAM: 3.33 GB


## Creating a Large List: Initial Memory Allocation

Now we'll create a list with 20 million integers. Few facts:

1. **Memory per element**: Each integer in Python uses more memory than you might expect.
2. **List overhead**: Python lists store pointers (references) to objects, not the objects themselves.
3. **Contiguous pointers**: The pointers are stored one-by-one in a contiguous block of memory, but the actual integer objects may be scattered throughout memory.
4. **Performance impact**: Accessing list elements is fast because pointer access is efficient, but scattered integer objects can lead to less efficient CPU caching compared to storing raw data contiguously.

In [4]:
big_list = list(range(20_000_000))

### Measuring List Memory Usage

The `sys.getsizeof()` function returns the size of the list object itself (the container), not including the memory used by the individual elements. For a list of integers, this measures:

- The list structure overhead
- Space for storing references/pointers to the integer objects
- Any over-allocated space for future growth

**Expected behavior**: A list of 20 million integers should use approximately 150-200 MB of RAM, significantly more than the theoretical minimum due to Python's object model.

In [5]:
print(f"Memory used by big_list: {sys.getsizeof(big_list) / (1024 ** 2):.3f} MB")

Memory used by big_list: 152.588 MB


## Understanding List Growth: The Append Operation

When we append elements to a Python list, the memory doesn't grow linearly. Instead, Python uses a smart over-allocation strategy:

### Python's Growth Strategy
- **Small lists**: Grow by 4 elements at a time
- **Medium lists**: Grow by approximately 1/8 of current size + 3
- **Large lists**: Grow by approximately 1/16 of current size + 3

This strategy ensures that:
1. **Amortized O(1) performance**: Most append operations are fast
2. **Memory efficiency**: Not too much memory is wasted
3. **Predictable behavior**: Growth pattern is consistent

Let's observe this behavior by appending elements one by one and watching when memory reallocations occur.

In [5]:
# add one integer at the end of the list, check its size and memory usage
big_list.append(1)
print(f"Memory used by big_list after appending one element: {sys.getsizeof(big_list) / (1024 ** 2):.3f} MB")

Memory used by big_list after appending one element: 171.661 MB


In [6]:
# add one integer at the end of the list, check its size and memory usage
big_list.append(1)
print(f"Memory used by big_list after appending one element: {sys.getsizeof(big_list) / (1024 ** 2):.3f} MB")

Memory used by big_list after appending one element: 171.661 MB


### Observing Memory Reallocation Pattern

The following loop demonstrates Python's list growth strategy in action. Notice that:

1. **Memory doesn't grow with every append**: Most operations reuse existing allocated space
2. **Growth happens in chunks**: When reallocation occurs, significant additional memory is allocated
3. **Growth rate decreases**: As the list gets larger, the relative growth rate decreases

**Warning**: This loop will double the size of your list, so it may take a while and use substantial memory!

In [7]:
for i in range(len(big_list)):
    size_of_big_list = sys.getsizeof(big_list)
    big_list.append(i)
    if size_of_big_list != sys.getsizeof(big_list):
        print(f"Memory used after appending element {i}: {sys.getsizeof(big_list) / (1024 ** 2):.3f} MB, added {(sys.getsizeof(big_list) - size_of_big_list) / 1024**2:.3f} MB")
        size_of_big_list = sys.getsizeof(big_list)

Memory used after appending element 2500002: 193.119 MB, added 21.458 MB
Memory used after appending element 5312506: 217.259 MB, added 24.140 MB
Memory used after appending element 8476574: 244.416 MB, added 27.157 MB
Memory used after appending element 12036150: 274.969 MB, added 30.552 MB
Memory used after appending element 16040674: 309.340 MB, added 34.371 MB


## Exercise: Growing a List to 10 GB

**Your Task**: Implement code that grows the `big_list` until it reaches approximately 10 GB of memory usage. Print the memory usage each time it increases significantly.

### What to Observe:
1. **Growth pattern**: How much memory is added with each reallocation?
2. **Performance**: Does the append operation get slower as the list grows?
3. **System impact**: Monitor your system's available memory during the process

### Implementation Hints:
- Use a while loop that continues until the list reaches 10 GB
- Track the previous memory size to detect when reallocations occur
- Consider the performance implications - this may take considerable time!
- Be prepared to interrupt the process if your system runs low on memory

### Safety Note:
**Caution**: This exercise will consume approximately 10 GB of RAM. Ensure your system has sufficient memory available before running this code!

## Writing Data to File: Memory vs Storage

This final section demonstrates writing our list to a CSV file. This operation highlights important concepts:

### Memory vs Disk Storage:
- **In-memory representation**: Our list occupies significant RAM due to Python object overhead
- **File representation**: Is it true that the same data requires much less disk space when stored as text ?
- **Serialization overhead**: Converting Python objects to text format takes time


In [9]:
with open('some.csv', 'w', newline='') as f:
    for row in big_list:
        f.write(f"{row}\n")

## Summary: Key Takeaways

Through this tutorial, you've learned about Python's list memory model:

### Key Insights:
1. **Over-allocation Strategy**: Python lists pre-allocate extra memory to minimize expensive reallocations
2. **Growth Pattern**: Lists grow by increasingly smaller relative amounts as they get larger
3. **Memory Overhead**: Python objects have significant memory overhead compared to raw data
4. **Performance Trade-offs**: Memory over-allocation trades space for time efficiency

### Practical Implications:
- **Large datasets**: Consider alternatives like NumPy arrays or databases for very large datasets
- **Memory planning**: Account for Python's memory overhead when estimating resource requirements

### Next Steps:
In the following notebooks, we'll explore more memory-efficient alternatives like NumPy arrays and learn about tools like Dask for handling datasets that don't fit in memory.